In [1]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

# from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors
import xesmf as xe
from defs import regularAtlmask

In [2]:
datafolder = ''

In [3]:
experiment = 'Eoi400' 
model = 'CCSM4-UoT'
folder = datafolder+'Raw_data/'+model+'/'

In [4]:
if experiment == 'E280':
    ds = xr.open_dataset(folder+experiment+'/E280.nc')
    dsV = xr.open_dataset(folder+experiment+'/E280.nc')
    dsT = xr.open_dataset(folder+experiment+'/E280.nc')
    dsS = xr.open_dataset(folder+experiment+'/E280.nc')
    dstos = xr.open_dataset(folder+experiment+'/tos_Omon_UofT-CCSM4_piControl_r1i1p1f1_gr1_160101-170012.nc')
else:
    ds = xr.open_dataset(folder+experiment+'/Eoi400.nc')
    dsT = xr.open_dataset(folder+experiment+'/Eoi400.nc')
    dsV = xr.open_dataset(folder+experiment+'/Eoi400.nc')
    dsS = xr.open_dataset(folder+experiment+'/Eoi400.nc')
    dstos = xr.open_dataset(folder+experiment+'/tos_Omon_UofT-CCSM4_midPliocene-eoi400_r1i1p1f1_gr1_150101-160012.nc')
    

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'tos' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [54]:
dsS = dsS.drop(['ULAT','ULONG','lat_aux_grid','moc_z'])
 
regridder = xe.Regridder(dsS, dstos, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsS.SALT)

ds = xr.Dataset(data_vars = dict(so=(("z","lat","lon"), var)), coords = dict(z = var.z_t.values/100, lat = var.lat[:,0].values, lon = var.lon[0,:].values))
xr.Dataset.to_netcdf(ds, path = datafolder+'/Processed/'+model+'/'+experiment+'/so_1x1_100yr.nc')

In [55]:
dsV = dsV.drop(['TLAT','TLONG','lat_aux_grid','moc_z'])
 
regridder = xe.Regridder(dsV, dstos, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsV.VVEL)

ds = xr.Dataset(data_vars = dict(vo=(("z","lat","lon"), var)), coords = dict(z = var.z_t.values/100, lat = var.lat[:,0].values, lon = var.lon[0,:].values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/vo_1x1_100yr.nc')

In [56]:
dsT = dsT.drop(['ULAT','ULONG','lat_aux_grid','moc_z'])
 
regridder = xe.Regridder(dsT, dstos, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsT.TEMP)

ds = xr.Dataset(data_vars = dict(thetao=(("z","lat","lon"), var)), coords = dict(z = var.z_t.values/100, lat = var.lat[:,0].values, lon = var.lon[0,:].values))
xr.Dataset.to_netcdf(ds, path = datafolder+'/Processed/'+model+'/'+experiment+'/thetao_1x1_100yr.nc')